In [31]:
import openpyxl
from openpyxl import Workbook
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.workbook.defined_name import DefinedName

In [32]:
import os
import requests
import zipfile
import io

# Visuals in Excel: Setup styling parameters

In [33]:

# Define fill colors for Category cells (only for Column A in MetaDataTemplate)
color_map = {
    "Study Information": "FFC7CE",         # light red
    "Sample Information": "C6EFCE",          # light green
    "Experimental Conditions": "FFC7CE",     # light red
    "Sequencing Details": "C6EFCE",          # light green
    "Bioinformatics Analysis": "FFC7CE",     # light red
    "Ethical & Legal": "C6EFCE",             # light green
    "Contact Information": "FFC7CE"          # light red
}

# Header styling for both sheets
header_fill = PatternFill(start_color="FFC000", end_color="FFC000", fill_type="solid")
header_font = Font(bold=True)
default_alignment = Alignment(wrap_text=True)
# Light grey font for Example_Value column (but now we apply drop-down to Value column, so we won't change its font)
light_grey_font = Font(color="808080")

# Uniform column width
col_width = 25

# 1. Create the workbook and the MetaDataTemplate sheet

In [34]:
wb = Workbook()

# Create "MetaDataTemplate" sheet.
metadata_ws = wb.active
metadata_ws.title = "MetaDataTemplate"
metadata_header = [
    "Category", 
    "Key", 
    "Value", 
    "Example_Value", 
    "Description", 
    "Controlled_Vocabulary/Value-Restrictions", 
    "Resource_Link(s)"
]
metadata_ws.append(metadata_header)

# Apply header formatting for MetaDataTemplate.
for cell in metadata_ws[1]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = default_alignment

# Define metadata rows.
metadata_rows = [
    ["Study Information", "Title", "", "Analysis of Intestinal Crypt, Villus, and Polyp Cells", "A concise title describing the overall study", "Free text; consider limiting length (e.g., ≤250 characters)", "N/A"],
    ["Study Information", "Summary/Abstract", "", "This study investigates transcriptomic changes under infection conditions.", "Detailed study summary and objectives", "Free text; follow standard writing guidelines", "N/A"],
    ["Study Information", "Experimental_Design", "", "RNA-seq of single cells comparing infected vs. control conditions.", "Overview of study design including factors, controls, and replicates", "Free text; consider using standardized descriptors if available", "https://www.ebi.ac.uk/arrayexpress/help/"],
    ["Sample Information", "Sample_Identifier", "", "RNA-Seq_Sample_001", "Unique identifier for each biological sample", "Must be unique; use an alphanumeric/underscore format (e.g., regex: ^[A-Za-z0-9_]+$)", "N/A"],
    ["Sample Information", "Organism", "", "Pseudomonas aeruginosa", "Species name from which the sample is derived", "Must conform to NCBI Taxonomy names", "https://www.ncbi.nlm.nih.gov/Taxonomy/taxonomyhome.html"],
    ["Sample Information", "Tissue", "", "Pancreas", "Specific tissue", "Use controlled terms from ontologies such as Uberon or Cell Ontology", "http://www.ontobee.org/"],
    ["Sample Information", "Cell_Type", "", "HeLa cells", "Specific cell line, or cell type of origin", "Use controlled terms from ontologies such as Uberon or Cell Ontology", "http://www.ontobee.org/"],
    ["Sample Information", "Strain_Identifier", "", "JCM 14847", "Strain information (if applicable)", "Use recognized strain IDs (e.g., from ATCC, DSMZ)", "https://www.atcc.org/"],
    ["Sample Information", "Collection_Date", "", "2025-03-24", "Date on which the sample was collected", "Must follow ISO 8601 format (YYYY-MM-DD)", "https://www.iso.org/iso-8601-date-and-time-format.html"],
    ["Experimental Conditions", "Treatment_Conditions", "", "Anti-Grem1 antibody treatment", "Description of the experimental treatment(s) or conditions applied", "If available, select from a controlled list (e.g., EFO terms)", "https://www.ebi.ac.uk/efo/"],
    ["Experimental Conditions", "Treatment_Duration", "", "10 weeks", "Duration for which the treatment was applied", "Numeric value with a unit; standardize unit selection (e.g., hours, days, weeks)", "N/A"],
    ["Experimental Conditions", "Concentration", "", "30 mg/kg", "Concentration or dosage of the treatment agent", "Numeric value with unit; ensure consistency across samples", "N/A"],
    ["Sequencing Details", "Library_Preparation_Method", "", "TruSeq Stranded mRNA Library Prep Kit", "Method or kit used for nucleic acid library preparation", "Use a predefined list from vendor documentation or internal standards", "https://www.illumina.com/"],
    ["Sequencing Details", "Sequencing_Platform", "", "Illumina NovaSeq", "Instrument or platform used for sequencing", "Must be selected from a controlled list (e.g., Illumina, PacBio, Oxford Nanopore)", "https://www.ncbi.nlm.nih.gov/sra/docs/submitdesign/"],
    ["Sequencing Details", "Sequencing_Depth", "", "50 million reads", "Number of reads obtained or overall depth of sequencing", "Free numeric text; standard units should be used", "N/A"],
    ["Sequencing Details", "Single_or_Paired-End", "", "Paired-end", "Specifies whether the sequencing was single-end or paired-end", "Allowed values: Single-end, Paired-end", "N/A"],
    ["Sequencing Details", "Instrument_Model", "", "Illumina HiSeq 2000", "Specific model of the sequencing instrument", "Use a controlled list (as provided by the manufacturer)", "https://www.illumina.com/systems/sequencing-platforms.html"],
    ["Sequencing Details", "Quality_Control_Metrics", "", "RIN > 8, Q30 > 85%", "Key quality metrics such as RNA integrity number and quality scores", "Free numeric values; standardize units and thresholds", "N/A"],
    ["Bioinformatics Analysis", "Read_Alignment_Algorithm", "", "HISAT2", "Software used for aligning reads to the reference genome", "Choose from a controlled list (e.g., HISAT2, STAR, Bowtie2)", "https://github.com/ewels/MultiQC"],
    ["Bioinformatics Analysis", "Genome_Reference", "", "Pseudomonas aeruginosa PAO1", "Reference genome or assembly used for analysis", "Must match identifiers from recognized genome databases", "https://www.ncbi.nlm.nih.gov/assembly/"],
    ["Bioinformatics Analysis", "Differential_Expression_Analysis", "", "DESeq2 with FDR < 0.05", "Software and statistical thresholds used for differential expression analysis", "Choose from accepted tools (e.g., DESeq2, edgeR, limma)", "https://bioconductor.org/packages/release/bioc/html/DESeq2.html"],
    ["Bioinformatics Analysis", "Gene_Annotation", "", "Ensembl, KEGG", "Databases or methods used for gene functional annotation", "Use controlled database names; if possible, select from a predefined list", "https://www.ensembl.org/"],
    ["Ethical & Legal", "Ethical_Approval", "", "IRB #12345", "Details regarding ethics committee or IRB approval", "Free text; may follow a predefined institutional format", "N/A"],
    ["Ethical & Legal", "Data_Sharing_Policy", "", "Data available under CC-BY 4.0", "Conditions or restrictions related to data sharing", "Must be selected from a controlled list of licensing options", "https://creativecommons.org/licenses/by/4.0/"],
    ["Contact Information", "Contributor/Contact_Name", "", "Jane Doe", "Primary contact or contributor for the study", "Free text; consider enforcing a 'Last Name, First Name' format", "N/A"],
    ["Contact Information", "Contact_Email", "", "jane.doe@example.com", "Email address of the study contact", "Must follow standard email format", "https://html.spec.whatwg.org/multipage/input.html#valid-e-mail-address"],
    ["Contact Information", "Contact_Affiliation", "", "Example University", "Affiliation or institution of the study contact", "Free text; optionally choose from a controlled list if available", "https://grid.ac/"]
]

# Append metadata rows and apply formatting.
for row in metadata_rows:
    metadata_ws.append(row)
    current_row = metadata_ws.max_row
    # Color only the Category cell (Column A) based on the category.
    cat_value = metadata_ws.cell(row=current_row, column=1).value
    if cat_value in color_map:
        metadata_ws.cell(row=current_row, column=1).fill = PatternFill(start_color=color_map[cat_value],
                                                                        end_color=color_map[cat_value],
                                                                        fill_type="solid")
    # Set wrap text for each cell in the row.
    for cell in metadata_ws[current_row]:
        cell.alignment = default_alignment

# Set column widths for MetaDataTemplate sheet.
for col in metadata_ws.columns:
    col_letter = col[0].column_letter
    metadata_ws.column_dimensions[col_letter].width = col_width




# 2. Create the Database sheet with controlled vocabulary values

In [35]:

db_ws = wb.create_sheet(title="Database")
db_header = [
    "Title",
    "Summary/Abstract",
    "Experimental Design",
    "Sample Identifier",
    "Organism",
    "Tissue/Cell Type",
    "Strain Identifier",
    "Collection Date",
    "Treatment Conditions",
    "Treatment Duration",
    "Concentration",
    "Library Preparation Method",
    "Sequencing Platform",
    "Sequencing Depth",
    "Single or Paired-End",
    "Instrument Model",
    "Quality Control Metrics",
    "Read Alignment Algorithm",
    "Genome Reference",
    "Differential Expression Analysis",
    "Gene Annotation",
    "Ethical Approval",
    "Data Sharing Policy",
    "Contributor/Contact Name",
    "Contact Email",
    "Contact Affiliation"
]
db_ws.append(db_header)

# Apply header formatting for Database.
for cell in db_ws[1]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = default_alignment

# Set column widths for Database sheet.
for col in db_ws.columns:
    col_letter = col[0].column_letter
    db_ws.column_dimensions[col_letter].width = col_width





## 2.1 download data and fill database with values

###  2.1.1 Example

In [36]:
# Fill in controlled vocabulary values for demonstration:
# For "Organism" (Column 5)
organism_values = ["Homo sapiens", "Pseudomonas aeruginosa", "Mus musculus"]
start_row = 2
for i, value in enumerate(organism_values, start=start_row):
    db_ws.cell(row=i, column=5, value=value).alignment = default_alignment

# For "Single or Paired-End" (Column 15)
spe_values = ["Single-end", "Paired-end"]
for i, value in enumerate(spe_values, start=start_row):
    db_ws.cell(row=i, column=15, value=value).alignment = default_alignment

###  2.1.1 Organism

In [37]:

# -------------------------------
# Step 1. Download and filter taxdump organism names
# -------------------------------
zip_filename = "taxdmp.zip"
taxdump_url = "https://ftp.ncbi.nih.gov/pub/taxonomy/taxdmp.zip"

# Check for local cache.
if os.path.exists(zip_filename):
    print(f"Using cached file {zip_filename}")
    with open(zip_filename, "rb") as f:
        zip_content = f.read()
else:
    print("Downloading taxdmp.zip from NCBI...")
    response = requests.get(taxdump_url)
    if response.status_code != 200:
        print("Error downloading the file:", response.status_code)
        exit(1)
    zip_content = response.content
    with open(zip_filename, "wb") as f:
        f.write(zip_content)

# Extract and filter scientific names from names.dmp.
all_scientific_names = []
with zipfile.ZipFile(io.BytesIO(zip_content)) as z:
    files = z.namelist()
    if "names.dmp" not in files:
        print("names.dmp not found in the zip file.")
    else:
        with z.open("names.dmp") as f:
            for line in f:
                line = line.decode("utf-8").strip()
                if not line:
                    continue
                # Each line is delimited by "|". Expected columns:
                # tax_id | name_txt | unique name | name class |
                parts = [part.strip() for part in line.split("|")]
                if len(parts) < 4:
                    continue
                if parts[3] == "scientific name":
                    all_scientific_names.append(parts[1])

print(f"Total scientific names found: {len(all_scientific_names)}")

# Define keywords relevant to infectious diseases.
keywords = [
    "virus", "bacter", "fung", "parasite", "protozoa",
    "mycobacterium", "salmonella", "staphylococcus", "streptococcus",
    "pseudomonas", "influenza", "hepatitis"
]

# Filter names using keywords (case-insensitive).
filtered_names = [name for name in all_scientific_names 
                  if any(kw.lower() in name.lower() for kw in keywords)]
# Get unique, sorted names.
unique_filtered_names = sorted(set(filtered_names))
print(f"Number of filtered (unique) scientific names: {len(unique_filtered_names)}")



# --- Database sheet ---
db_ws = wb.create_sheet(title="Database")
db_header = [
    "Title",
    "Summary/Abstract",
    "Experimental Design",
    "Sample Identifier",
    "Organism",
    "Tissue/Cell Type",
    "Strain Identifier",
    "Collection Date",
    "Treatment Conditions",
    "Treatment Duration",
    "Concentration",
    "Library Preparation Method",
    "Sequencing Platform",
    "Sequencing Depth",
    "Single or Paired-End",
    "Instrument Model",
    "Quality Control Metrics",
    "Read Alignment Algorithm",
    "Genome Reference",
    "Differential Expression Analysis",
    "Gene Annotation",
    "Ethical Approval",
    "Data Sharing Policy",
    "Contributor/Contact Name",
    "Contact Email",
    "Contact Affiliation"
]
db_ws.append(db_header)

# Apply header formatting for Database sheet.
header_fill = PatternFill(start_color="FFC000", end_color="FFC000", fill_type="solid")
header_font = Font(bold=True)
default_alignment = Alignment(wrap_text=True)
for cell in db_ws[1]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = default_alignment

# Set uniform column widths for Database sheet.
col_width = 25
for col in db_ws.columns:
    col_letter = col[0].column_letter
    db_ws.column_dimensions[col_letter].width = col_width

# Add filtered organism values to the "Organism" column (Column E) in the Database sheet.
start_row = 2
for i, value in enumerate(unique_filtered_names, start=start_row):
    db_ws.cell(row=i, column=5, value=value).alignment = default_alignment

print("Filtered organism names have been added to the Database sheet, column 'Organism'.")

# -------------------------------
# Step 3. Create named ranges for data validation (if needed)
# -------------------------------
# Here you might want to create named ranges using a subset of these organisms.
# For example, to create a named range "OrganismList" covering all organisms in Database column E:
end_row = start_row + len(unique_filtered_names) - 1
organism_range = f"Database!$E${start_row}:$E${end_row}"
wb.defined_names.add(DefinedName('OrganismList', attr_text=organism_range))

# Save the workbook.
output_filename = "combined_metadata_template.xlsx"
wb.save(output_filename)
print(f"Excel file saved as '{output_filename}'.")


Using cached file taxdmp.zip
Total scientific names found: 2656022
Number of filtered (unique) scientific names: 503442
Filtered organism names have been added to the Database sheet, column 'Organism'.
Excel file saved as 'combined_metadata_template.xlsx'.


# 3. Add Data Validation (Drop-down Menus) to the MetaDataTemplate sheet using named ranges

In [39]:
# ================================================
# 3. Add Data Validation (Drop-down Menus) to the MetaDataTemplate sheet using named ranges
# ================================================


# Assume 'unique_filtered_names' is the list of filtered organism names
start_row = 2
end_row = start_row + len(unique_filtered_names) - 1
organism_range = f"Database!$E${start_row}:$E${end_row}"
wb.defined_names.add(DefinedName('OrganismList', attr_text=organism_range))


# Create a named range "SPEList" for Database!O2:O3.
spe_range = "Database!$O$2:$O$3"
wb.defined_names.add(DefinedName('SPEList', attr_text=spe_range))


# Drop-down for "Organism" in MetaDataTemplate.
# Find the row where the "Key" (Column B) equals "Organism" and apply the drop-down on the "Value" (Column C).
organism_row = None
for row in metadata_ws.iter_rows(min_row=2, max_row=metadata_ws.max_row, values_only=False):
    if row[1].value == "Organism":
        organism_row = row[0].row  # Get the row number
        break

if organism_row:
    cell_coord = f"C{organism_row}"  # Drop-down in the Value column (Column C)
    organism_validation = DataValidation(
        type="list",
        formula1="=OrganismList",  # Reference the named range
        allow_blank=True,
        showDropDown=False
    )
    metadata_ws.add_data_validation(organism_validation)
    organism_validation.add(cell_coord)

# Drop-down for "Single_or_Paired-End" in MetaDataTemplate.
# Find the row where the "Key" (Column B) equals "Single_or_Paired-End" and apply the drop-down on the "Value" (Column C).
single_paired_row = None
for row in metadata_ws.iter_rows(min_row=2, max_row=metadata_ws.max_row, values_only=False):
    if row[1].value == "Single_or_Paired-End":
        single_paired_row = row[0].row
        break

if single_paired_row:
    cell_coord = f"C{single_paired_row}"  # Drop-down in the Value column (Column C)
    spe_validation = DataValidation(
        type="list",
        formula1="=SPEList",  # Reference the named range for Single or Paired-End
        allow_blank=True,
        showDropDown=False
    )
    metadata_ws.add_data_validation(spe_validation)
    spe_validation.add(cell_coord)


# 4. Save the workbook to a file

In [30]:

output_filename = "combined_metadata_template3.xlsx"
wb.save(output_filename)
print(f"Excel file with 'MetaDataTemplate' and 'Database' sheets created as '{output_filename}'")

Excel file with 'MetaDataTemplate' and 'Database' sheets created as 'combined_metadata_template3.xlsx'
